In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [ ]:
#Scrapes IMDb website
#categories is just used to scrape pages
categories = ['Action', 'Drama', 'Crime', 'Adventure', 'Comedy', 'Romance','Horror','Animation','History','Western','Family']#,'War','Mystery','Western','History','Family','Animation','Sport','Film-Noir','Musical',]
#arrays for dataframe columns
summaries = []
genre1 = []
genre2 = []
genre3 = []
titles = []
ratingavgs = []
ratings = []
y = 1 #just for counting the amount of movies 
pagenum = input("Enter the number of pages to search through (Note: Every page is 1050 entries, but there may not be 1050 entries x pages in the finalized dataset): ")
count = int(input("What should be the max count of genre?: "))

#categories for scraping pages of "genres" listed
for category in categories:

  URL = f"https://www.imdb.com/search/title/?title_type=feature&genres={category}&start=1"

  for i in range(int(pagenum)):
    headers = {'Accept-Language': 'en-US,en;q=0.8'} #do this so it doesn't get Chinese imdb info
    page = requests.get(URL, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    movies = soup.find_all('div', class_="lister-item-content")

    #Parses through the blocks of movie titles
    for movie in movies:
      title = movie.find('h3', class_="lister-item-header")
      title = title.text.strip().split("\n")[1]

      ratingavg = movie.find('div', class_="inline-block ratings-imdb-rating")

      genre = movie.find('span', class_="genre")
      genrelist = genre.text.strip().replace(' ','').split(",")
      votes = movie.find('p', class_="sort-num_votes-visible")


      #Checks and avoids duplicates and no ratings
      if (title not in titles) and (votes != None) and (ratingavg != None) and (genre1.count(genrelist[0]) < count):

        #if maxamount == True;

        titles.append(title)

        #Scrapes the rating
        #ratingavg = movie.find('div', class_="inline-block ratings-imdb-rating")
        ratingavg = ratingavg.find('strong').text.strip()
        ratingavgs.append(ratingavg)

        #Scrapes amount of ratings
        votes = movie.find('p', class_="sort-num_votes-visible").find_all('span')[1]
        votes = votes.text.split()
        ratings.append(int(votes[0].replace(',','')))

        #Scrapes the genre
        #genre = movie.find('span', class_="genre")
        #genrelist = genre.text.strip().replace(' ','').split(",")
        genre1.append(genrelist[0])

        #If there's only 1 genre, make the 2nd genre blank
        if len(genrelist) > 1:
          genre2.append(genrelist[1])
        else:
          genre2.append(" ")

        #If there's only 2 genres, make the 3rd genre blank
        if len(genrelist) > 2:
          genre3.append(genrelist[2])
        else:
          genre3.append(" ")

        print(f'{y}: {title}')
        y = y + 1

        #theres multiple of the same class name so find_all instead of find
        summary = movie.find_all('p', class_="text-muted")[1]
        summaries.append(summary.text.strip())

        nextpage = soup.find('a', class_='lister-page-next next-page')
        #print(test['href'])
        URL = 'https://www.imdb.com' + nextpage['href']




#Creates dataframe
dfzip = list(zip(titles, summaries, genre1, genre2, genre3, ratingavgs, ratings))
movieset = pd.DataFrame(dfzip, columns = ['titles', 'summaries','genre1', 'genre2', 'genre3', 'average rating','num of ratings'])
movieset.to_csv("movieset.csv", index = False)

In [ ]:
movieset['genre1'].value_counts()